In [2]:
# DONE

# FT-HMC implemented for 8x8 2D QED (using SiLU as activation function).

# Try to minimize size of the force in training. No significant improvements.

# Some test on ergodicity
# (calculate the probablity of generating the configs obtained via conventional HMC).

# TODO

# Plot the force size distribution
# Is the large force from the original action or Field-Transformation the determinant?
# If from the determinant, then the fermion force won't cause problem for HMC

# Use the same Field-Transformation for larger system (say 16x16, 32x32, 64x64, etc)
# Study how the delta H depends on the system size ( perhaps delta H ~ sqrt(volume) )

# Study the auto-correlation for observables, topo, plaq, flowed plaq, etc.

# Improving the Field-Transformation to reduce force.

In [3]:
import torch
import math
import sys
import os
from timeit import default_timer as timer
from functools import reduce
from field_transformation import *

In [4]:
# From Xiao-Yong

class Param:
    def __init__(
            self,
            beta: float = 6.0,
            lat: list = [64, 64],
            tau: float = 2.0,
            nstep: int = 50,
            ntraj: int = 256,
            nrun: int = 4,
            nprint: int = 256,
            seed: int = 11*13,
            randinit: bool = False,
            nth: int = int(os.environ.get('OMP_NUM_THREADS', '2')),
            nth_interop: int = 2
    ):
        self.params = {
            'beta': beta, 
            'lat': lat,
            'nd': len(lat),
            'volume': reduce(lambda x, y: x * y, lat),
            'tau': tau,
            'nstep': nstep,
            'dt': tau / nstep,
            'ntraj': ntraj,
            'nrun': nrun,
            'nprint': nprint,
            'seed': seed,
            'randinit': randinit,
            'nth': nth,
            'nth_interop': nth_interop,
        }
        for k, v in self.params.items():
            setattr(self, k, v)
            
    def initializer(self) -> torch.Tensor:
        if self.randinit:
            return torch.empty((param.nd,) + param.lat).uniform_(-math.pi, math.pi)
        else:
            return torch.zeros((param.nd,) + param.lat)
        
    def summary(self) -> str:
        return '\n'.join([f'{k}: {v}' for k, v in self.params.items()])
    
    def uniquestr(self) -> str:
        lat = ".".join(str(x) for x in self.lat)
        return f"out_l{lat}_b{param.beta}_n{param.ntraj}_t{param.tau}_s{param.nstep}.out"

    
def action(param: Param, f: torch.Tensor) -> torch.Tensor:
    return (-param.beta)*torch.sum(torch.cos(plaqphase(f)))


def force(param: Param, f: torch.Tensor) -> torch.Tensor:
    f.requires_grad_(True)
    s = action(param, f)
    f.grad = None
    s.backward()
    ff = f.grad
    f.requires_grad_(False)
    return ff


def regularize(f):
    p2 = 2*math.pi
    f_ = (f - math.pi) / p2
    return p2*(f_ - torch.floor(f_) - 0.5)

def plaqphase(f: torch.Tensor) -> torch.Tensor:
    return (f[0, :]
            - f[1, :]
            - torch.roll(f[0, :], shifts=-1, dims=1)
            + torch.roll(f[1, :] ,shifts=-1, dims=0))


def topocharge(f: torch.Tensor) -> torch.Tensor:
    return torch.floor(0.1 + torch.sum(regularize(plaqphase(f))) / (2 * math.pi))

#plaqphase = lambda f: f[0,:] - f[1,:] - torch.roll(f[0,:], shifts=-1, dims=1) + torch.roll(f[1,:], shifts=-1, dims=0)
#topocharge = lambda f: torch.floor(0.1 + torch.sum(regularize(plaqphase(f))) / (2*math.pi))


def leapfrog(param: Param, x: torch.Tensor, p: torch.Tensor) -> (torch.Tensor, torch.Tensor):
    dt = param.dt
    x_ = x + 0.5*dt*p
    f = force(param, x_)
    p_ = p + (-dt)*f
    print(f'plaq(x) {action(param, x) / (-param.beta*param.volume)}  force.norm {torch.linalg.norm(f)}')
    for i in range(param.nstep-1):
        x_ = x_ + dt*p_
        p_ = p_ + (-dt)*force(param, x_)
    x_ = x_ + 0.5*dt*p_
    return (x_, p_)

def hmc(param: Param, x: torch.Tensor):
    p = torch.randn_like(x)
    act0 = action(param, x) + 0.5*torch.sum(p*p)
    x_, p_ = leapfrog(param, x, p)
    xr = regularize(x_)
    act = action(param, xr) + 0.5*torch.sum(p_*p_)
    prob = torch.rand([], dtype=torch.float64)
    dH = act-act0
    exp_mdH = torch.exp(-dH)
    acc = prob < exp_mdH
    newx = xr if acc else x
    return (dH, exp_mdH, acc, newx)

put = lambda s: sys.stdout.write(s)

In [5]:
param = Param(
    beta = 2.0,
    lat = (8, 8),
    tau = 2, # 0.3
    nstep = 8, # 3
    # ADJUST ME
    ntraj = 2, # 2**16 # 2**10 # 2**15
    #
    nprint = 2,
    seed = 1331)

In [6]:
torch.manual_seed(param.seed)

torch.set_num_threads(param.nth)
torch.set_num_interop_threads(param.nth_interop)
os.environ["OMP_NUM_THREADS"] = str(param.nth)
os.environ["KMP_BLOCKTIME"] = "0"
os.environ["KMP_SETTINGS"] = "1"
os.environ["KMP_AFFINITY"]= "granularity=fine,verbose,compact,1,0"

torch.set_default_tensor_type(torch.DoubleTensor)

In [7]:
def run(param, field = param.initializer()):
    with open(param.uniquestr(), "w") as O:
        params = param.summary()
        O.write(params)
        put(params)
        plaq, topo = (action(param, field) / (-param.beta*param.volume), topocharge(field))
        status = f"Initial configuration:  plaq: {plaq}  topo: {topo}\n"
        O.write(status)
        put(status)
        ts = []
        for n in range(param.nrun):
            t = -timer()
            for i in range(param.ntraj):
                dH, exp_mdH, acc, field = hmc(param, field)
                plaq = action(param, field) / (-param.beta*param.volume)
                topo = topocharge(field)
                ifacc = "ACCEPT" if acc else "REJECT"
                status = f"Traj: {n*param.ntraj+i+1:4}  {ifacc}:  dH: {dH:< 12.8}  exp(-dH): {exp_mdH:< 12.8}  plaq: {plaq:< 12.8}  topo: {topo:< 3.3}\n"
                O.write(status)
                if (i+1) % (param.ntraj//param.nprint) == 0:
                    put(status)
            t += timer()
            ts.append(t)
        print("Run times: ", ts)
        print("Per trajectory: ", [t/param.ntraj for t in ts])
    return field

In [8]:
field = run(param)
field_run = torch.reshape(field,(1,)+field.shape)

beta: 2.0
lat: (8, 8)
nd: 2
volume: 64
tau: 2
nstep: 8
dt: 0.25
ntraj: 2
nrun: 4
nprint: 2
seed: 1331
randinit: False
nth: 2
nth_interop: 2Initial configuration:  plaq: 1.0  topo: 0.0
plaq(x) 1.0  force.norm 9.246826593536493
Traj:    1  ACCEPT:  dH: -2.1286122    exp(-dH):  8.4031965    plaq:  0.87165922   topo:  0.0
plaq(x) 0.8716592163190823  force.norm 13.600657198197021
Traj:    2  ACCEPT:  dH: -0.43065065   exp(-dH):  1.5382581    plaq:  0.85113562   topo:  0.0
plaq(x) 0.8511356194331359  force.norm 17.543097274383317
Traj:    3  ACCEPT:  dH: -0.52286113   exp(-dH):  1.686847     plaq:  0.81229714   topo:  0.0
plaq(x) 0.8122971355705814  force.norm 17.22642099298651
Traj:    4  ACCEPT:  dH:  0.039753441  exp(-dH):  0.96102636   plaq:  0.78167924   topo:  0.0
plaq(x) 0.7816792404187667  force.norm 17.046687492063338
Traj:    5  ACCEPT:  dH:  0.42993024   exp(-dH):  0.65055447   plaq:  0.80392524   topo:  0.0
plaq(x) 0.803925239737245  force.norm 17.257422008841356
Traj:    6  ACCE

In [9]:
def ft_flow(flow, f):
    for layer in flow:
        f, lJ = layer.forward(f)
    return f.detach()

def ft_flow_inv(flow, f):
    for layer in reversed(flow):
        f, lJ = layer.reverse(f)
    return f.detach()

def ft_action(param, flow, f):
    y = f
    logJy = 0.0
    for layer in flow:
        y, lJ = layer.forward(y)
        logJy += lJ
    action = U1GaugeAction(param.beta)
    s = action(y) - logJy
    return s

def ft_force(param, flow, field, create_graph = False):
    # f is the field follows the transformed distribution (close to prior distribution)
    f = field
    f.requires_grad_(True)
    s = ft_action(param, flow, f)
    ss = torch.sum(s)
    # f.grad = None
    ff, = torch.autograd.grad(ss, f, create_graph = create_graph)
    f.requires_grad_(False)
    return ff

In [10]:
def train_step(model, action, optimizer, metrics, batch_size, with_force = False, pre_model = None):
    layers, prior = model['layers'], model['prior']
    optimizer.zero_grad()
    #
    xi = None
    if pre_model != None:
        pre_layers, pre_prior = pre_model['layers'], pre_model['prior']
        pre_xi = pre_prior.sample_n(batch_size)
        x = ft_flow(pre_layers, pre_xi)
        xi = ft_flow_inv(layers, x)
    #
    xi, x, logq = apply_flow_to_prior(prior, layers, batch_size=batch_size, xi=xi)
    logp = -action(x)
    #
    force_size = torch.tensor(0.0)
    dkl = calc_dkl(logp, logq)
    loss = torch.tensor(0.0)
    if with_force:
        assert pre_model != None
        force = ft_force(param, layers, xi, True)
        force_size = torch.sum(torch.square(force))
        loss = force_size
    else:
        loss = dkl
    #
    loss.backward()
    #
    # minimization target
    # loss mini
    # -> (logq - logp) mini
    # -> (action - logJ) mini
    #
    optimizer.step()
    ess = compute_ess(logp, logq)
    #
    print(grab(loss),
          grab(force_size),
          grab(dkl),
          grab(ess),
          torch.linalg.norm(ft_force(param, layers, xi)))
    #
    metrics['loss'].append(grab(loss))
    metrics['force'].append(grab(force_size))
    metrics['dkl'].append(grab(dkl))
    metrics['logp'].append(grab(logp))
    metrics['logq'].append(grab(logq))
    metrics['ess'].append(grab(ess))

def flow_train(param, with_force = False, pre_model = None):  # packaged from original ipynb by Xiao-Yong Jin
    # Theory
    lattice_shape = param.lat
    link_shape = (2,*param.lat)
    beta = param.beta
    u1_action = U1GaugeAction(beta)
    # Model
    prior = MultivariateUniform(torch.zeros(link_shape), 2*np.pi*torch.ones(link_shape))
    #
    n_layers = 24
    n_s_nets = 2
    hidden_sizes = [8,8]
    kernel_size = 3
    layers = make_u1_equiv_layers(lattice_shape=lattice_shape, n_layers=n_layers, n_mixture_comps=n_s_nets,
                                  hidden_sizes=hidden_sizes, kernel_size=kernel_size)
    set_weights(layers)
    model = {'layers': layers, 'prior': prior}
    # Training
    base_lr = .001
    optimizer = torch.optim.Adam(model['layers'].parameters(), lr=base_lr)
    optimizer_wf = torch.optim.Adam(model['layers'].parameters(), lr=base_lr / 100.0)
    #
    # ADJUST ME
    N_era = 10
    N_epoch = 100
    #
    batch_size = 64
    print_freq = N_epoch # epochs
    plot_freq = 1 # epochs
    history = {
        'loss' : [],
        'force' : [],
        'dkl' : [],
        'logp' : [],
        'logq' : [],
        'ess' : []
    }
    for era in range(N_era):
        for epoch in range(N_epoch):
            train_step(model, u1_action, optimizer, history, batch_size)
            if with_force:
                train_step(model, u1_action, optimizer_wf, history, batch_size,
                           with_force = with_force, pre_model = pre_model)
            if epoch % print_freq == 0:
                print_metrics(history, print_freq, era, epoch)
    return model,u1_action

def flow_eval(model, u1_action):  # packaged from original ipynb by Xiao-Yong Jin
    ensemble_size = 1024
    u1_ens = make_mcmc_ensemble(model, u1_action, 64, ensemble_size)
    print("Accept rate:", np.mean(u1_ens['accepted']))
    Q = grab(topo_charge(torch.stack(u1_ens['x'], axis=0)))
    X_mean, X_err = bootstrap(Q**2, Nboot=100, binsize=16)
    print(f'Topological susceptibility = {X_mean:.2f} +/- {X_err:.2f}')
    print(f'... vs HMC estimate = 1.23 +/- 0.02')

In [11]:
pre_flow_model, flow_act = flow_train(param)
flow_eval(pre_flow_model,flow_act)
pre_flow = pre_flow_model['layers']

-235.80379178382827 0.0 -235.80379178382827 0.015742986900554163 tensor(173.9276)
== Era 0 | Epoch 0 metrics ==
	loss -235.804
	force 0
	dkl -235.804
	logp 0.693854
	logq -235.11
	ess 0.015743
-240.22592204900428 0.0 -240.22592204900428 0.01726504117436026 tensor(169.3064)
-241.73488535507386 0.0 -241.73488535507386 0.016400911777968756 tensor(167.0606)
-247.46169379805445 0.0 -247.46169379805445 0.015810153849540865 tensor(160.3539)
-253.25240939507336 0.0 -253.25240939507336 0.01941417680419557 tensor(158.7390)
-256.75166216575127 0.0 -256.75166216575127 0.015804690643426364 tensor(154.5671)
-257.21331835493976 0.0 -257.21331835493976 0.01562887239859159 tensor(157.1432)
-261.58546505634735 0.0 -261.58546505634735 0.020297799438260253 tensor(151.7423)
-265.36695333901866 0.0 -265.36695333901866 0.0156412599546812 tensor(151.7542)
-266.2862499543668 0.0 -266.2862499543668 0.016146700456890064 tensor(155.4196)
-270.4000967148002 0.0 -270.4000967148002 0.01831122343147412 tensor(156.129

	loss -283.91
	force 0
	dkl -283.91
	logp 79.3769
	logq -204.533
	ess 0.084167
-285.2184109390458 0.0 -285.2184109390458 0.05831390274229266 tensor(174.0637)
-285.5080370194683 0.0 -285.5080370194683 0.1166042724279074 tensor(158.2435)
-285.47454534289994 0.0 -285.47454534289994 0.08551370972479476 tensor(159.2134)
-285.7410910826486 0.0 -285.7410910826486 0.09694266245844649 tensor(166.2664)
-285.1816577242461 0.0 -285.1816577242461 0.18581720592138307 tensor(160.6161)
-284.94563297692713 0.0 -284.94563297692713 0.037887675923891334 tensor(165.5799)
-285.1367904117073 0.0 -285.1367904117073 0.07095008093136997 tensor(161.8093)
-284.88992314549574 0.0 -284.88992314549574 0.1292122816474257 tensor(175.4481)
-285.48417859246416 0.0 -285.48417859246416 0.09883028991631716 tensor(170.3434)
-285.2071173702128 0.0 -285.2071173702128 0.1686882618690228 tensor(159.7044)
-285.5761087567976 0.0 -285.5761087567976 0.10562976216060008 tensor(153.3455)
-284.9573830256865 0.0 -284.9573830256865 0.16

-286.07403823706 0.0 -286.07403823706 0.12732676696077913 tensor(180.7658)
-286.171818284237 0.0 -286.171818284237 0.11671141404654953 tensor(164.3912)
-286.32278366467307 0.0 -286.32278366467307 0.06557855207649757 tensor(159.9655)
-286.29907929594674 0.0 -286.29907929594674 0.24660392220817695 tensor(178.4996)
-286.70345074852213 0.0 -286.70345074852213 0.13025084497793202 tensor(180.4461)
-286.229829868461 0.0 -286.229829868461 0.20246256551857741 tensor(171.2551)
-286.182029290086 0.0 -286.182029290086 0.15544772488884223 tensor(171.7185)
-286.30195926420197 0.0 -286.30195926420197 0.20824601214876662 tensor(164.9257)
-286.82556491817616 0.0 -286.82556491817616 0.29026847375309905 tensor(173.1734)
-286.4177581006933 0.0 -286.4177581006933 0.10261035785936544 tensor(182.5192)
-286.3569199373462 0.0 -286.3569199373462 0.0577708900064954 tensor(164.2266)
-286.1784712944642 0.0 -286.1784712944642 0.09900306937592586 tensor(172.7846)
-286.4338164251218 0.0 -286.4338164251218 0.091641668

-286.8060090744058 0.0 -286.8060090744058 0.2206663872752518 tensor(205.1283)
-286.83962092058255 0.0 -286.83962092058255 0.04996853536265053 tensor(330.5075)
-286.5782347705788 0.0 -286.5782347705788 0.25769687839519223 tensor(173.5427)
-286.43768491517295 0.0 -286.43768491517295 0.24694264303141616 tensor(202.9833)
-285.99569953934156 0.0 -285.99569953934156 0.0933217723408387 tensor(196.0997)
-286.8836865054619 0.0 -286.8836865054619 0.32105523564432786 tensor(172.0458)
-286.92523266636414 0.0 -286.92523266636414 0.11408531772523961 tensor(236.2626)
-286.37424945860846 0.0 -286.37424945860846 0.2763150618731418 tensor(187.5516)
-286.87764579347015 0.0 -286.87764579347015 0.33560313755030724 tensor(196.6000)
-286.6968894822108 0.0 -286.6968894822108 0.18031224420547132 tensor(187.4605)
-286.52879833557733 0.0 -286.52879833557733 0.20499660157025143 tensor(196.2932)
-286.2364603749641 0.0 -286.2364603749641 0.11616869499488437 tensor(197.9263)
-286.1971812829619 0.0 -286.1971812829619

-286.7255404396324 0.0 -286.7255404396324 0.19201121471792307 tensor(196.1057)
-287.095822172987 0.0 -287.095822172987 0.39830166341459383 tensor(269.2700)
-286.7159158974782 0.0 -286.7159158974782 0.2398544135059376 tensor(147.8612)
-286.7530398847164 0.0 -286.7530398847164 0.2174537179489148 tensor(221.9993)
-287.1781980115127 0.0 -287.1781980115127 0.35783872935491234 tensor(205.1628)
-286.6681289032741 0.0 -286.6681289032741 0.29862014031078504 tensor(216.4736)
-287.1086413244841 0.0 -287.1086413244841 0.3646640687135799 tensor(225.3921)
-287.08025553959646 0.0 -287.08025553959646 0.1936124729578828 tensor(220.9878)
-286.90318376541006 0.0 -286.90318376541006 0.32413694928566456 tensor(176.0489)
-287.09745699479805 0.0 -287.09745699479805 0.23504139883141362 tensor(225.3163)
-286.69981513238446 0.0 -286.69981513238446 0.22369218917289793 tensor(208.3010)
-287.0663898890353 0.0 -287.0663898890353 0.3056273431823325 tensor(175.1973)
-286.76778890105044 0.0 -286.76778890105044 0.32448

In [12]:
flow_model, flow_act = flow_train(param, with_force=True, pre_model=pre_flow_model)
flow_eval(flow_model,flow_act)
flow = flow_model['layers']
# flow.eval()

-238.39049113095712 0.0 -238.39049113095712 0.020874773022601868 tensor(172.2697)
19895.576947002137 19895.576947002137 -319.57948662292415 0.05410863666028212 tensor(140.8972)
== Era 0 | Epoch 0 metrics ==
	loss 9828.59
	force 9947.79
	dkl -278.985
	logp 45.1002
	logq -233.885
	ess 0.0374917
-239.99976242090156 0.0 -239.99976242090156 0.01601788104416998 tensor(168.6970)
18428.018472041527 18428.018472041527 -316.4374867879671 0.039221438631699995 tensor(135.6055)
-243.9512480907071 0.0 -243.9512480907071 0.026686704957950145 tensor(165.5903)
15839.57981419014 15839.57981419014 -315.55972312953156 0.0188816702314684 tensor(125.7213)
-245.25210238388522 0.0 -245.25210238388522 0.052730437139897154 tensor(160.9210)
14641.889907644227 14641.889907644227 -311.52194899679625 0.019416117340973465 tensor(120.8860)
-251.90357941579376 0.0 -251.90357941579376 0.015625031459330835 tensor(156.8555)
12966.220321964054 12966.220321964054 -309.63083721817384 0.05373812992987525 tensor(113.7596)
-25

-282.0288759674046 0.0 -282.0288759674046 0.01748891782679761 tensor(172.2337)
18107.09874547043 18107.09874547043 -291.0170564260919 0.09874884956579436 tensor(134.5299)
-282.2200999082913 0.0 -282.2200999082913 0.06634073337839831 tensor(177.6308)
17836.32625000032 17836.32625000032 -290.4221557180066 0.05567670330681381 tensor(133.5189)
-282.6662888984654 0.0 -282.6662888984654 0.11425298353406703 tensor(175.1554)
17472.065803800302 17472.065803800302 -290.81753088904554 0.043110457357864354 tensor(132.1487)
-281.94873595086125 0.0 -281.94873595086125 0.041115689491939764 tensor(183.0737)
18391.90620942812 18391.90620942812 -291.12573669849246 0.016070010122786912 tensor(135.5751)
-282.77547573857044 0.0 -282.77547573857044 0.05110075496229343 tensor(176.0552)
19292.295820321247 19292.295820321247 -291.48025269768925 0.04186817312021927 tensor(138.8499)
-283.03733035669813 0.0 -283.03733035669813 0.04628325185417098 tensor(178.1560)
23552.63477779828 23552.63477779828 -291.086571454

20215.922302307892 20215.922302307892 -289.0490789642482 0.08581517078753059 tensor(142.0087)
-284.7708420011242 0.0 -284.7708420011242 0.1241824655435952 tensor(145.8697)
19158.524028265674 19158.524028265674 -289.117127331115 0.06887955802300738 tensor(138.2079)
-285.0769650030387 0.0 -285.0769650030387 0.052996048514764 tensor(152.4883)
22839.25626319651 22839.25626319651 -289.01215128805075 0.0850424860735054 tensor(150.9762)
-285.02814523391044 0.0 -285.02814523391044 0.11159996303823944 tensor(155.1642)
20938.334217736752 20938.334217736752 -288.77423971463463 0.08875526203928062 tensor(144.5194)
-285.43705250318334 0.0 -285.43705250318334 0.021591439216530797 tensor(144.5166)
18380.643497061435 18380.643497061435 -288.8009548812096 0.08265548621339397 tensor(135.4307)
-285.13135757909276 0.0 -285.13135757909276 0.08273905264687126 tensor(162.2770)
19564.29309519095 19564.29309519095 -288.17521469352937 0.1997791956284614 tensor(139.7457)
-284.9900123827216 0.0 -284.9900123827216

-285.772797657745 0.0 -285.772797657745 0.13352600059287248 tensor(157.7767)
20738.191999614173 20738.191999614173 -288.56340876681264 0.2610773608756829 tensor(143.7585)
-285.46334408463053 0.0 -285.46334408463053 0.15066971058082745 tensor(157.9661)
22300.80950093914 22300.80950093914 -288.40516639264354 0.18582107082424165 tensor(149.1505)
-285.3310271813408 0.0 -285.3310271813408 0.14386454062725842 tensor(142.6556)
24843.47600563333 24843.47600563333 -287.6717669867619 0.05836557620280894 tensor(157.4783)
-285.5410839964654 0.0 -285.5410839964654 0.09045148083147031 tensor(168.7516)
21417.12381737355 21417.12381737355 -288.60570552724664 0.18768438021406503 tensor(146.1840)
-285.730168523674 0.0 -285.730168523674 0.122192229778682 tensor(164.3966)
19133.649837078316 19133.649837078316 -288.0868052717084 0.2116947041712562 tensor(138.2011)
-285.398444279386 0.0 -285.398444279386 0.12247826272055558 tensor(166.5890)
24505.120501426667 24505.120501426667 -288.6896016102867 0.13146141

23795.474775542818 23795.474775542818 -288.5274882315637 0.30356889931007763 tensor(154.1901)
-285.1827729262832 0.0 -285.1827729262832 0.1418946311663951 tensor(156.6378)
19224.02100400642 19224.02100400642 -288.45599867016364 0.18968129677743464 tensor(138.3854)
-285.62606895193437 0.0 -285.62606895193437 0.13258059759868943 tensor(142.7041)
23667.233065038257 23667.233065038257 -288.40961342577515 0.23209298321446936 tensor(153.6850)
-286.16013444276166 0.0 -286.16013444276166 0.07365081620139523 tensor(150.4744)
23860.893357346416 23860.893357346416 -288.1094493766308 0.23214431501453328 tensor(154.2875)
-286.28434058454536 0.0 -286.28434058454536 0.03286456625991591 tensor(149.8403)
21377.70386144462 21377.70386144462 -288.2114292755558 0.16811685408311078 tensor(146.0131)
-285.729869331477 0.0 -285.729869331477 0.15966489171558237 tensor(154.8468)
20664.923273788314 20664.923273788314 -287.60827147142953 0.07925116611560677 tensor(143.6191)
-285.73469490586814 0.0 -285.7346949058

...

27958.50679232544 27958.50679232544 -288.2966551439023 0.19286580868432462 tensor(163.9601)
-286.4797627522033 0.0 -286.4797627522033 0.07811362030583703 tensor(148.1422)
21180.912273406586 21180.912273406586 -288.44936874868677 0.13943360775931582 tensor(145.4348)
-286.69214649501464 0.0 -286.69214649501464 0.14031344264756274 tensor(164.3497)
27653.80453501371 27653.80453501371 -288.3503238926605 0.16379168719307263 tensor(166.1107)
-286.5229748137177 0.0 -286.5229748137177 0.22328707916552662 tensor(172.0494)
18978.2745603812 18978.2745603812 -288.3035356350615 0.10711937408197916 tensor(136.8900)
-286.70210836875356 0.0 -286.70210836875356 0.1702354599587314 tensor(175.4810)
25303.834752597635 25303.834752597635 -288.1242508819921 0.21995941519325043 tensor(158.6727)
-286.2409051241623 0.0 -286.2409051241623 0.23110730884735647 tensor(157.8309)
27144.042387531506 27144.042387531506 -287.69811209288355 0.131790883572475 tensor(164.1271)
-286.33128420486446 0.0 -286.33128420486446 0.

33331.56520296979 33331.56520296979 -288.4050308632264 0.17093248168197042 tensor(181.7066)
-286.8974465791176 0.0 -286.8974465791176 0.13211741495537271 tensor(139.9096)
25372.704086255522 25372.704086255522 -288.37268731161976 0.37642936973324176 tensor(158.9987)
-286.33816760643793 0.0 -286.33816760643793 0.06978035473281015 tensor(145.1814)
23048.127015965718 23048.127015965718 -287.99142862212256 0.2365390862535243 tensor(151.7385)
-286.5838858475015 0.0 -286.5838858475015 0.1649489493516032 tensor(173.0642)
20378.00255407058 20378.00255407058 -287.8234318608211 0.1646989551977042 tensor(142.4871)
-286.1246728293569 0.0 -286.1246728293569 0.1364022705656823 tensor(165.5897)
21544.541008429558 21544.541008429558 -288.06438290411745 0.27149724495031774 tensor(146.6611)
-286.37468364734957 0.0 -286.37468364734957 0.15329447533337204 tensor(155.0557)
23742.087118479503 23742.087118479503 -288.3184926304711 0.31780992051977874 tensor(151.1523)
-286.5063451750442 0.0 -286.5063451750442 

26391.745446696055 26391.745446696055 -288.3340861235174 0.2883365073684297 tensor(162.0164)
-286.68894503008664 0.0 -286.68894503008664 0.08670579283613039 tensor(167.5100)
23883.661724018868 23883.661724018868 -288.5054464286691 0.1801344826650062 tensor(154.4185)
-286.5927184516541 0.0 -286.5927184516541 0.11679488193891184 tensor(200.7802)
21293.329420524162 21293.329420524162 -287.8666124551879 0.30063998766202715 tensor(145.6931)
-286.37298988082 0.0 -286.37298988082 0.35194058324364674 tensor(197.5455)
29343.66251115444 29343.66251115444 -288.24526837225295 0.25323359190072553 tensor(169.3390)
-286.5957784599935 0.0 -286.5957784599935 0.24908419820003672 tensor(142.1839)
31947.409620594757 31947.409620594757 -288.12750083430115 0.10406359526193001 tensor(177.8800)
-286.76380623734565 0.0 -286.76380623734565 0.22083220448735955 tensor(156.8642)
32450.10986093465 32450.10986093465 -288.1204881208329 0.2831497826651419 tensor(179.5333)
-286.80472023674383 0.0 -286.80472023674383 0.

-286.23514710809434 0.0 -286.23514710809434 0.1400074389289302 tensor(178.7152)
23049.86885887923 23049.86885887923 -288.03295312951195 0.26476172590038954 tensor(151.5239)
-286.87773890589233 0.0 -286.87773890589233 0.19954941237977275 tensor(185.7843)
36904.57836125448 36904.57836125448 -287.8699683752664 0.16980275901617972 tensor(190.8515)
-286.98102191102254 0.0 -286.98102191102254 0.21878809772352076 tensor(146.6684)
56865.21121008001 56865.21121008001 -288.0733563950155 0.3976256351523512 tensor(229.3597)
-286.9387012004107 0.0 -286.9387012004107 0.28772970247960034 tensor(160.7471)
37897.010232110835 37897.010232110835 -288.3071493993833 0.15932416654930243 tensor(191.2314)
-286.53212349034357 0.0 -286.53212349034357 0.26896251001767374 tensor(220.9754)
31088.38097107392 31088.38097107392 -288.1497600924496 0.23119466352553444 tensor(175.1177)
-286.8379147595121 0.0 -286.8379147595121 0.3398327070781594 tensor(190.7170)
30926.305707327072 30926.305707327072 -287.6669524075952 0

29545.839601755917 29545.839601755917 -288.4465730929619 0.47428446769830945 tensor(171.6324)
-286.8151212339759 0.0 -286.8151212339759 0.273896469550297 tensor(178.4583)
33188.38437750572 33188.38437750572 -288.42573690510267 0.2661439264663623 tensor(181.3265)
-286.8457000405217 0.0 -286.8457000405217 0.4055879646505006 tensor(175.7673)
26187.467106153097 26187.467106153097 -287.764546675734 0.19408951557778947 tensor(160.5654)
-286.5367729033023 0.0 -286.5367729033023 0.2349166899233862 tensor(175.3840)
29040.124023825392 29040.124023825392 -288.0832697460643 0.16832868328932768 tensor(169.9603)
-286.3983647482503 0.0 -286.3983647482503 0.06131807077005022 tensor(173.6732)
30794.413639064853 30794.413639064853 -287.8960162582364 0.2992857938128901 tensor(175.4131)
-286.6032350169363 0.0 -286.6032350169363 0.28029650946655027 tensor(163.7994)
38401.75597004997 38401.75597004997 -288.22061612992644 0.21854761268187697 tensor(195.4205)
-286.44760726806146 0.0 -286.44760726806146 0.1997

In [10]:
def test_force(x = None):
    model = flow_model
    layers, prior = model['layers'], model['prior']
    if x == None:
        pre_model = pre_flow_model
        pre_layers, pre_prior = pre_model['layers'], pre_model['prior']
        pre_xi = pre_prior.sample_n(1)
        x = ft_flow(pre_layers, pre_xi)
    xi = ft_flow_inv(layers, x)
    f = ft_force(param, layers, xi)
    f_s = torch.linalg.norm(f)
    print(f_s)

test_force()
test_force(field_run)

tensor(20.7829)
tensor(24.9007)


In [11]:
field_run = run(param, field_run[0])
field_run = torch.reshape(field_run,(1,)+field_run.shape)

latsize = (8, 8)
volume = 64
beta = 2.0
trajs = 2
tau = 2
steps = 8
seed = 1331
nth = 2
nth_interop = 2
Initial configuration:  plaq: 0.6627686419055754  topo: 0.0
plaq(x) 0.6627686419055754  force.norm 21.091837451367958
Traj:    1  ACCEPT:  dH: -0.49041166   exp(-dH):  1.6329883    plaq:  0.62473448   topo:  2.0
plaq(x) 0.6247344841194027  force.norm 21.8840629070871
Traj:    2  ACCEPT:  dH:  0.66073851   exp(-dH):  0.51646978   plaq:  0.66851193   topo:  0.0
plaq(x) 0.6685119259671379  force.norm 19.583396204466677
Traj:    3  REJECT:  dH:  0.62826715   exp(-dH):  0.5335155    plaq:  0.66851193   topo:  0.0
plaq(x) 0.6685119259671379  force.norm 19.540474594479193
Traj:    4  REJECT:  dH:  0.25176081   exp(-dH):  0.77743067   plaq:  0.66851193   topo:  0.0
plaq(x) 0.6685119259671379  force.norm 18.73962959721593
Traj:    5  ACCEPT:  dH:  0.26645544   exp(-dH):  0.76609014   plaq:  0.73566717   topo:  1.0
plaq(x) 0.735667166271016  force.norm 18.993473339448784
Traj:    6  ACCEPT:  d

In [133]:
flows = flow

print(f'plaq(field_run[0]) {action(param, field_run[0]) / (-param.beta*param.volume)}')
# field.requires_grad_(True)
x = field_run
logJ = 0.0
for layer in reversed(flows):
    x, lJ = layer.reverse(x)
    logJ += lJ

# x is the prior distribution now
    
x.requires_grad_(True)
    
y = x
logJy = 0.0
for layer in flows:
    y, lJ = layer.forward(y)
    logJy += lJ
    
s = action(param, y[0]) - logJy

print(logJ,logJy)


# print("eff_action", s + 136.3786)

print("original_action", action(param, y[0]) + 91)

print("eff_action", s + 56)

s.backward()

f = x.grad

x.requires_grad_(False)

print(f'plaq(x) {action(param, x[0]) / (-param.beta*param.volume)}  logJ {logJ}  force.norm {torch.linalg.norm(f)}')

print(f'plaq(y) {action(param, y[0]) / (-param.beta*param.volume)}')

print(f'plaq(x) {action(param, field_run[0]) / (-param.beta*param.volume)}  force.norm {torch.linalg.norm(force(param, field_run[0]))}')


plaq(field_run[0]) 0.5729559235604118
tensor([18.9736], grad_fn=<AddBackward0>) tensor([-18.9736], grad_fn=<AddBackward0>)
original_action tensor(17.6616, grad_fn=<AddBackward0>)
eff_action tensor([1.6352], grad_fn=<AddBackward0>)
plaq(x) -0.20566029157065324  logJ tensor([18.9736], grad_fn=<AddBackward0>)  force.norm 15.38544434986163
plaq(y) 0.5729560026167126
plaq(x) 0.5729559235604118  force.norm 20.91876473600617


In [13]:
print(x.shape)
x = ft_flow_inv(flow, field_run)
# x = field_run
#for layer in reversed(flows):
#    x, lJ = layer.reverse(x)
ff = ft_force(param, flow, x)
print(torch.linalg.norm(ff))
fff = ft_force(param, flow, x)
print(torch.linalg.norm(fff))

torch.Size([1, 2, 8, 8])
tensor(30.5185)
tensor(30.5185)


In [14]:
x = ft_flow_inv(flow, field_run)
ft_action(param, flow, x)

tensor([-54.6079], grad_fn=<SubBackward0>)

In [110]:
def flattern(l):
    return [x for y in l for x in y]

def average(l):
    return sum(l) / len(l)

def sub_avg(l):
    avg = average(l)
    return np.array([x - avg for x in l])

In [116]:
ft_hmc_info_list = []
def ft_leapfrog(param, flow, x, p):
    mom_norm = torch.sum(p*p)
    info_list = []
    dt = param.dt
    x_ = x + 0.5*dt*p
    f = ft_force(param, flow, x_)
    p_ = p + (-dt)*f
    info = np.array((float(torch.linalg.norm(f)),
                     float(ft_action(param, flow, x_).detach()),
                     float(torch.sum(p*p_)/np.sqrt(mom_norm*torch.sum(p_*p_)))))
    info_list.append(info)
    for i in range(param.nstep-1):
        x_ = x_ + dt*p_
        f = ft_force(param, flow, x_)
        info = np.array((float(torch.linalg.norm(f)),
                        float(ft_action(param, flow, x_).detach()),
                        float(torch.sum(p*p_)/np.sqrt(mom_norm*torch.sum(p_*p_)))))
        info_list.append(info)
        p_ = p_ + (-dt)*f
    x_ = x_ + 0.5*dt*p_
    print(np.sqrt(average([l[0]**2 for l in info_list])),
          (info_list[0][1], info_list[-1][1]),
          info_list[-1][2])
    ft_hmc_info_list.append(info_list)
    return (x_, p_)

def ft_hmc(param, flow, field):
    x = ft_flow_inv(flow, field)
    p = torch.randn_like(x)
    act0 = ft_action(param, flow, x).detach() + 0.5*torch.sum(p*p)
    x_, p_ = ft_leapfrog(param, flow, x, p)
    xr = regularize(x_)
    act = ft_action(param, flow, xr).detach() + 0.5*torch.sum(p_*p_)
    prob = torch.rand([], dtype=torch.float64)
    dH = act-act0
    exp_mdH = torch.exp(-dH)
    acc = prob < exp_mdH
    # ADJUST ME
    newx = xr if acc else x
    # newx = xr
    newfield = ft_flow(flow, newx)
    return (float(dH), float(exp_mdH), acc, newfield)

In [125]:
def ft_run(param, flow, field = None):
    if field == None:
        field = param.initializer()
    ft_hmc_info_list = []
    with open(param.uniquestr(), "w") as O:
        params = param.summary()
        O.write(params)
        put(params)
        plaq, topo = (action(param, field) / (-param.beta*param.volume), topocharge(field))
        status = f"Initial configuration:  plaq: {plaq}  topo: {topo}\n"
        O.write(status)
        put(status)
        ts = []
        for n in range(param.nrun):
            t = -timer()
            for i in range(param.ntraj):
                field_run = torch.reshape(field,(1,)+field.shape)
                dH, exp_mdH, acc, field_run = ft_hmc(param, flow, field_run)
                field = field_run[0]
                plaq = action(param, field) / (-param.beta*param.volume)
                topo = topocharge(field)
                ifacc = "ACCEPT" if acc else "REJECT"
                status = f"Traj: {n*param.ntraj+i+1:4}  {ifacc}:  dH: {dH:< 12.8}  exp(-dH): {exp_mdH:< 12.8}  plaq: {plaq:< 12.8}  topo: {topo:< 3.3}\n"
                O.write(status)
                if (i+1) % (param.ntraj//param.nprint) == 0:
                    put(status)
            t += timer()
            ts.append(t)
        print("Run times: ", ts)
        print("Per trajectory: ", [t/param.ntraj for t in ts])
    return field

In [159]:
param = Param(
    beta = 2.0,
    lat = (8, 8),
    tau = 0.5, # 0.3
    nstep = 64, # 3
    # ADJUST ME
    ntraj = 4, # 2**16 # 2**10 # 2**15
    nprint = 4,
    #
    seed = 1331)

# field = ft_run(param, pre_flow)
field = ft_run(param, pre_flow, field)

latsize = (8, 8)
volume = 64
beta = 2.0
trajs = 4
tau = 0.5
steps = 64
seed = 1331
nth = 2
nth_interop = 2
Initial configuration:  plaq: 0.7629951284651968  topo: -1.0
13.22268982123488 (-53.34181291527922, -54.876554449401496) 0.9655721914591343
Traj:    1  ACCEPT:  dH:  0.0030688103  exp(-dH):  0.99693589   plaq:  0.68926726   topo: -1.0
21.852915596175038 (-54.85201954833743, -52.21010307281895) 0.9534584595204836
Traj:    2  ACCEPT:  dH: -0.079341423  exp(-dH):  1.0825739    plaq:  0.63487933   topo: -2.0
20.332651317477275 (-52.29965736350433, -52.47600352890689) 0.9622497587200374
Traj:    3  ACCEPT:  dH: -0.048030786  exp(-dH):  1.049203     plaq:  0.69466522   topo: -1.0
27.52295044775975 (-52.42782795609963, -53.22876146049952) 0.9555116848770403
Traj:    4  ACCEPT:  dH:  1.252823     exp(-dH):  0.28569713   plaq:  0.70510825   topo:  0.0
21.40765677353727 (-53.19172805413828, -54.22214120009521) 0.9576481950578482
Traj:    5  ACCEPT:  dH: -0.0062113797  exp(-dH):  1.0062307  

In [151]:
param = Param(
    beta = 2.0,
    lat = (8, 8),
    tau = 0.5, # 0.3
    nstep = 64, # 3
    # ADJUST ME
    ntraj = 4, # 2**16 # 2**10 # 2**15
    nprint = 4,
    #
    seed = 1331)

# field = ft_run(param, pre_flow)
field = ft_run(param, pre_flow, field)

latsize = (8, 8)
volume = 64
beta = 2.0
trajs = 4
tau = 0.5
steps = 64
seed = 1331
nth = 2
nth_interop = 2
Initial configuration:  plaq: 0.635313317826161  topo: 0.0
20.78557074806181 (-50.70482292494995, -53.69532464920362) 0.94443367886955
Traj:    1  ACCEPT:  dH: -0.1493157    exp(-dH):  1.1610395    plaq:  0.66644803   topo:  1.0
28.214570380777243 (-53.64952198450136, -54.10207440344347) 0.9293616738812421
Traj:    2  ACCEPT:  dH:  0.096385415  exp(-dH):  0.90811395   plaq:  0.76428727   topo:  1.0
18.847970377810025 (-54.09168794183857, -54.316912234755506) 0.9535372516104789
Traj:    3  ACCEPT:  dH:  0.13176509   exp(-dH):  0.87654688   plaq:  0.74819831   topo:  0.0
17.4003441477011 (-54.33298820802238, -56.12852759824926) 0.9407703828248026
Traj:    4  ACCEPT:  dH: -0.13929567   exp(-dH):  1.1494639    plaq:  0.75147884   topo:  0.0
16.556162048299715 (-56.11146213578058, -54.08786736404058) 0.977222502981708
Traj:    5  ACCEPT:  dH: -0.010062502  exp(-dH):  1.0101133    plaq:

In [157]:
action_list = np.array([l[1] for l in flattern(ft_hmc_info_list)])
action_list = sub_avg(action_list)
np.sqrt(average(action_list**2))

1.3476780398265165

In [158]:
force_list = np.array([l[0] for l in flattern(ft_hmc_info_list)])
np.sqrt(average(force_list**2))

24.652070381082513

In [156]:
print(np.array(force_list[0:300]))

[ 17.32528814  24.41830328  35.86739086  40.38043113  28.76035006
  23.42855236  29.58922539  33.54458561  33.65143712  33.86083941
  33.49291895  31.83862211  27.47076189  24.25968801  22.01808465
  18.63787514  14.72785278  10.78893991   9.04838945  12.19247229
  15.60845521  16.04484037  14.48102181  13.42641911  14.66221209
  17.15532298  17.43832809  12.67743819   6.41088996  12.52344938
  15.78009459  10.51769265   8.67199085  11.21120182  12.65614775
  12.97190556  12.88566952  12.88993966  12.04653558   9.08958067
  17.7111367   14.50481818   7.32864662   8.0942277    7.76810601
   7.34379062   7.18410535   7.13059923   7.06343922   6.96975024
   6.95233258   7.24697015   8.13272086   9.6659413   11.37533083
  12.02592045 431.99648976  17.3099667   18.31988376  18.31030973
  17.84059728  17.20681364  16.7225185   14.38948063   6.68696119
   5.84353042   6.36965709   7.5283624    8.78319064  10.00651765
  11.24919959  12.34706306  12.60897163  13.51418753  24.01165632
  31.29971